In [3]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten, Add, InputLayer
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf

gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=1)
tf.test.gpu_device_name()

'/device:GPU:0'

In [12]:
ImageHeight = 256
ImageWidth = 256
ArchSelect = 'VGG16'

### Build Structure of Neural Network

In [5]:
if ArchSelect == 'VGG16':
    print('VGG16 Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', input_shape=(ImageHeight, ImageWidth, 1), activation='relu'))
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(Conv2D(filters=512 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=256))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

elif ArchSelect == 'LeNet':
    print('LeNet Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=6, kernel_size=(5,5), input_shape=(ImageHeight, ImageWidth,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=16, kernel_size=(5,5), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=120))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=84))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

else:
    print('Regular CNN Structure : ')
    model = Sequential()
    model.add(Conv2D(filters=64 ,kernel_size=(3,3), padding='same', input_shape=(ImageHeight, ImageWidth,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=128 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Conv2D(filters=256 ,kernel_size=(3,3), padding='same', activation='relu'))
    model.add(MaxPooling2D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(units=128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=128))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dense(units=5))
    model.add(Activation('softmax'))
    model.summary()

VGG16 Structure : 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 320, 320, 64)      640       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 320, 320, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 160, 160, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 160, 160, 128)     73856     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 160, 160, 128)     147584    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 80, 80, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 80, 80, 256)       29

### Load Train Data

In [6]:
import cv2 as cv
import gc
SampleNumber = 40
train_label = np.zeros(shape=(SampleNumber*5,), dtype = np.float16)


train_down = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('../down/down%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_down[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample] = 1

train_up = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('../up/up%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_up[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber] = 2

train_right = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('../right/right%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_right[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*2] = 3
    
train_left = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('../left/left%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_left[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*3] = 4

train_other = np.zeros(shape=(SampleNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(SampleNumber):
    img = cv.imread('../other/other%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_other[Sample] = img.reshape(-1, ImageHeight, ImageWidth, 1)
    train_label[Sample+SampleNumber*4] = 0
    
    
train_set = np.append(train_down, train_up ,axis=0)
train_set = np.append(train_set, train_right ,axis=0)
train_set = np.append(train_set, train_left ,axis=0)
train_set = np.append(train_set, train_other ,axis=0)
del train_down, train_up, train_left, train_right, train_other
gc.collect()

train_labels = np_utils.to_categorical(train_label)

print("train set shape :",train_set.shape)
print("train label shape :",train_labels.shape)
# down : 1, up : 2, right : 3, left : 4, other :0

train set shape : (200, 320, 320, 1)
train label shape : (200, 5)


### make sure train data

In [7]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from PIL import Image
plt.figure(figsize=(20,20))
plt.subplot(2,3,1)
img = Image.fromarray(train_set[0].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,2)
img = Image.fromarray(train_set[20].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,3)
img = Image.fromarray(train_set[40].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,4)
img = Image.fromarray(train_set[60].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,5)
img = Image.fromarray(train_set[80].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)
plt.subplot(2,3,6)
img = Image.fromarray(train_set[99].reshape(ImageHeight, ImageWidth).astype('uint8'))
plt.imshow(img)

### Training Model

In [8]:
batch_size = 5
epochs = 30

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4),metrics=['accuracy'])
history = model.fit(x=train_set, y=train_labels, batch_size=batch_size, epochs=epochs, verbose=1,shuffle=True, validation_split=0.2)

Train on 160 samples, validate on 40 samples
Epoch 1/30
160/160 [==============================] - 32s 198ms/step - loss: 0.8722 - acc: 0.6875 - val_loss: 5.9108 - val_acc: 0.0000e+00
Epoch 2/30
160/160 [==============================] - 18s 110ms/step - loss: 0.6314 - acc: 0.8000 - val_loss: 2.7105 - val_acc: 0.0750
Epoch 3/30
160/160 [==============================] - 17s 109ms/step - loss: 0.7323 - acc: 0.7125 - val_loss: 4.3159 - val_acc: 0.0000e+00
Epoch 4/30
160/160 [==============================] - 17s 109ms/step - loss: 0.5715 - acc: 0.8313 - val_loss: 5.9713 - val_acc: 0.0000e+00
Epoch 5/30
160/160 [==============================] - 17s 109ms/step - loss: 0.5407 - acc: 0.8188 - val_loss: 5.2844 - val_acc: 0.0000e+00
Epoch 6/30
160/160 [==============================] - 18s 110ms/step - loss: 0.3341 - acc: 0.8938 - val_loss: 5.4997 - val_acc: 0.0000e+00
Epoch 7/30
160/160 [==============================] - 17s 108ms/step - loss: 0.4234 - acc: 0.8750 - val_loss: 4.9815 - val_ac

KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
model.save('model_DHG_VGG16_224_S40.h5') 



### Load trained model to test

In [13]:
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Activation, Dense, Flatten, Add, InputLayer
from keras.optimizers import Adam
from keras.utils import np_utils
import tensorflow as tf
from keras.models import load_model
import cv2 as cv
model=load_model ('model_DHG_VGG16_224_S40.h5')

### Load test data

In [14]:
import cv2 as cv
import gc
testNumber = 20
train_label = np.zeros(shape=(SampleNumber*5,), dtype = np.float16)


train_test = np.zeros(shape=(testNumber,ImageHeight, ImageWidth, 1), dtype = np.float16)
for Sample in range(20,20+testNumber):
    img = cv.imread('../test/test%d'%Sample+'.jpg', cv.COLOR_BGR2GRAY)
    img = cv.resize(img,(ImageHeight,ImageWidth))
    train_test[Sample-20] = img.reshape(-1, ImageHeight, ImageWidth, 1)

### Prediction

In [15]:
pro = model.predict(train_test)
ans = np.argmax(pro,axis=1)
ans

array([0, 2, 3, 2, 2, 0, 2, 4, 3, 2, 4, 3, 2, 2, 2, 0, 4, 0, 0, 0],
      dtype=int64)

In [16]:
import pandas as pd
pro_df = pd.DataFrame(pro,columns=['no','down','up','right','left'])
pro_df

,no,down,up,right,left
0,0.820599,5.490410e-02,0.070715,0.023939,2.984322e-02
1,0.017461,3.106684e-03,0.978570,0.000644,2.181333e-04
2,0.454792,1.890147e-03,0.034467,0.506175,2.676944e-03
3,0.005415,7.112654e-03,0.986557,0.000650,2.652080e-04
4,0.337520,3.264005e-05,0.662130,0.000226,9.142770e-05
5,0.941359,1.811209e-02,0.037121,0.002664,7.439288e-04
6,0.003334,1.107642e-03,0.995059,0.000348,1.516946e-04
7,0.053959,2.195075e-03,0.019977,0.001795,9.220739e-01
8,0.302308,1.902724e-03,0.078200,0.615025,2.564786e-03
9,0.346743,8.375540e-02,0.552870,0.011815,4.817342e-03
